In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import cv2
import os
from PIL import Image
  
# Read the video from specified path
cam = cv2.VideoCapture("/content/DJI_0383.MP4")
  
try:
      
    # creating a folder named data
    if not os.path.exists('data'):
        os.makedirs('data')
        
# if not created then raise error
except OSError:
    print ('Error: Creating directory of data')
  
# frame
currentframe = 0
  
while(True):
      
    # reading from frame
    ret,frame = cam.read()
  
    if ret:
        # if video is still left continue creating images
        name = './data/frame' + str(currentframe) + '.jpg'
        print ('Creating...' + name)
  
        # writing the extracted images
      
        cv2.imwrite(name, frame)
  
        # increasing counter so that it will
        # show how many frames are created
        currentframe += 1
    else:
        break
  
# Release all space and windows once done


Creating..../data/frame0.jpg
Creating..../data/frame1.jpg
Creating..../data/frame2.jpg
Creating..../data/frame3.jpg
Creating..../data/frame4.jpg
Creating..../data/frame5.jpg
Creating..../data/frame6.jpg
Creating..../data/frame7.jpg
Creating..../data/frame8.jpg
Creating..../data/frame9.jpg
Creating..../data/frame10.jpg
Creating..../data/frame11.jpg
Creating..../data/frame12.jpg
Creating..../data/frame13.jpg
Creating..../data/frame14.jpg
Creating..../data/frame15.jpg
Creating..../data/frame16.jpg
Creating..../data/frame17.jpg
Creating..../data/frame18.jpg
Creating..../data/frame19.jpg
Creating..../data/frame20.jpg
Creating..../data/frame21.jpg
Creating..../data/frame22.jpg
Creating..../data/frame23.jpg
Creating..../data/frame24.jpg
Creating..../data/frame25.jpg
Creating..../data/frame26.jpg
Creating..../data/frame27.jpg
Creating..../data/frame28.jpg
Creating..../data/frame29.jpg
Creating..../data/frame30.jpg
Creating..../data/frame31.jpg
Creating..../data/frame32.jpg
Creating..../data/fr

In [2]:
import os
from PIL import Image

f = r'/content/data'      #Enter the location of your Image Folder
    
new_d = 64

for file in os.listdir(f):
    f_img = f+'/'+file
    try:
        img = Image.open(f_img)
        img = img.resize((new_d, new_d))
        img.save(f_img)
    except IOError:
        pass

In [ ]:
!unzip "/content/drive/MyDrive/new-aug-data.zip"

Streaming output truncated to the last 5000 lines.
  inflating: new-aug-data/new cashew data/training/healthy/healthy_original_healthy (1364).jpeg_938759b5-97c8-46e0-974f-ab1d3d253346.jpeg  
  inflating: new-aug-data/new cashew data/training/healthy/healthy_original_healthy (1364).jpg_e3b7027e-5a46-4acc-b4ee-732fe9473990.jpg  
  inflating: new-aug-data/new cashew data/training/healthy/healthy_original_healthy (1364).jpg_f3581ac6-9d1d-411c-817d-2e4d8d4df2f5.jpg  
  inflating: new-aug-data/new cashew data/training/healthy/healthy_original_healthy (1365).jpeg_b72e5451-4904-43ec-843c-6a52673f64cb.jpeg  
  inflating: new-aug-data/new cashew data/training/healthy/healthy_original_healthy (1366).jpeg_3e7b4efa-1753-461c-9854-db466e7bf051.jpeg  
  inflating: new-aug-data/new cashew data/training/healthy/healthy_original_healthy (1366).jpeg_43009472-0a8b-4baf-9318-02b2c3a2d815.jpeg  
  inflating: new-aug-data/new cashew data/training/healthy/healthy_original_healthy (1366).jpg_62ff4fc6-32b8-4d59

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import tensorflow_hub as hub
import os

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

In [4]:
root_dir = 'new-aug-data/new cashew data'
train_dir = os.path.join(root_dir, 'training')
test_dir = os.path.join(root_dir, 'testing')

In [5]:
module_selection = ("inception_v3", 299, 2048) #@param ["(\"mobilenet_v2\", 224, 1280)", "(\"inception_v3\", 299, 2048)"] {type:"raw", allow-input: true}
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/2".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
BATCH_SIZE = 64 #@param {type:"integer"}

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

In [7]:
do_data_augmentation = True #@param {type:"boolean"}
if do_data_augmentation:
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale = 1./255,
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, 
      height_shift_range=0.2,
      shear_range=0.2, 
      zoom_range=0.2,
      fill_mode='nearest', validation_split=0.2 )
else:
  train_datagen = validation_datagen

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    #validation_split=0.2,
    subset="training", 
    shuffle=True, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

FileNotFoundError: ignored

In [ ]:
# Inputs are suitably resized for the selected module.
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2)
validation_generator = validation_datagen.flow_from_directory(
    train_dir,
    subset="validation", 
    shuffle=False, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

Found 3277 images belonging to 2 classes.


In [ ]:
feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape=IMAGE_SIZE+(3,),
                                   output_shape=[FV_SIZE])

In [ ]:
do_fine_tuning = False #@param {type:"boolean"}
if do_fine_tuning:
  feature_extractor.trainable = True
  # unfreeze some layers of base network for fine-tuning
  for layer in feature_extractor.layers[-30:]:
      layer.trainable =True
  
else:
  feature_extractor.trainable = False

In [ ]:
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax',
                           kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              21802784  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 22,852,898
Trainable params: 1,050,114
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
#Compile model specifying the optimizer learning rate
LEARNING_RATE = 0.01
model.compile(
   optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE), 
   loss='categorical_crossentropy',
   metrics=['accuracy'])

In [ ]:

checkpoint_path = "training/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=1)

In [ ]:
print("train_generator.samples", train_generator.samples, ":", train_generator.batch_size, "train_generator.batch_size")
print("Hasil", train_generator.samples//train_generator.batch_size)
print("validation_generator.samples", validation_generator.samples, ":", validation_generator.batch_size, "validation_generator.batch_size")
print("Hasil", validation_generator.samples//validation_generator.batch_size)

train_generator.samples 13111 : 64 train_generator.batch_size
Hasil 204
validation_generator.samples 3277 : 64 validation_generator.batch_size
Hasil 51


In [ ]:
EPOCHS=25
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
STEPS_EPOCHS = train_generator.samples//train_generator.batch_size
VALID_STEPS=validation_generator.samples//validation_generator.batch_size
history = model.fit_generator( 
          train_generator,
          steps_per_epoch=STEPS_EPOCHS,
          epochs=EPOCHS,
          callbacks=[callback],
          validation_data=validation_generator,
          validation_steps=VALID_STEPS)

<ipython-input-17-d9da2e1345c3>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/25
204/204 [==============================] - 6616s 32s/step - loss: 0.0615 - accuracy: 0.9790 - val_loss: 0.0433 - val_accuracy: 0.9874
Epoch 2/25
204/204 [==============================] - 6626s 32s/step - loss: 0.0224 - accuracy: 0.9920 - val_loss: 0.0369 - val_accuracy: 0.9881
Epoch 3/25
204/204 [==============================] - 6476s 32s/step - loss: 0.0184 - accuracy: 0.9935 - val_loss: 0.0246 - val_accuracy: 0.9914
Epoch 4/25
204/204 [==============================] - 6459s 32s/step - loss: 0.0172 - accuracy: 0.9942 - val_loss: 0.0270 - val_accuracy: 0.9917
Epoch 5/25
204/204 [==============================] - 6448s 32s/step - loss: 0.0102 - accuracy: 0.9962 - val_loss: 0.0253 - val_accuracy: 0.9923
Epoch 6/25
204/204 [==============================] - 6507s 32s/step - loss: 0.0199 - accuracy: 0.9930 - val_loss: 0.0447 - val_accuracy: 0.9868


In [ ]:
  test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale = 1./255,
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, 
      height_shift_range=0.2,
      shear_range=0.2, 
      zoom_range=0.2,
      fill_mode='nearest', validation_split=0.2 )

In [ ]:
#test_subset_data_dir = "data/test_subset"

test_subset_generator = test_datagen.flow_from_directory(
test_dir,
batch_size = BATCH_SIZE,
target_size = IMAGE_SIZE,
class_mode = "categorical",
shuffle=False)

Found 3739 images belonging to 2 classes.


In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
Y_pred = model.predict_generator(test_subset_generator)
y_pred = np.argmax(Y_pred, axis=1)

img_nr = 0
for subdir, dirs, files in os.walk('test_dir'):
    for file in files:
        img_file = subdir + '/' + file
        image = load_img(img_file,target_size=(img_height,img_width))
        pred_emotion = category_names[y_pred[img_nr]]
        real_emotion = category_names[test_subset_generator.classes[img_nr]]
        plt.figure()
        plt.title('Predicted: ' + pred_emotion + '\n' + 'Actual:      ' + real_emotion)
        plt.imshow(image)
        img_nr = img_nr +1

<ipython-input-23-74c9149fe72d>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(test_subset_generator)


In [ ]:
Y_pred = model.predict_generator(test_subset_generator)
y_pred = np.argmax(Y_pred, axis=1)
accuracy = accuracy_score(test_subset_generator.classes, y_pred)
print("Accuracy in test set: %0.1f%% " % (accuracy * 100))

<ipython-input-24-9767efb362e1>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(test_subset_generator)


Accuracy in test set: 95.3% 


TENSORFLOWLITE CONVERSION

In [ ]:
tf.keras.models.save_model(model,'mobilenet_saved_model')

In [ ]:
new_model = tf.keras.models.load_model("mobilenet_saved_model")

In [ ]:
# convert the model to TFLite
!mkdir "tf_models"
TFLITE_MODEL = "tf_models/normal.tflite"


In [ ]:
# Get the concrete function from the Keras model.
run_model = tf.function(lambda x : new_model(x))

In [ ]:
# Save the concrete function.
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(new_model.inputs[0].shape, new_model.inputs[0].dtype)
)
     

In [ ]:
# Convert the model to standard TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converted_tflite_model = converter.convert()
open(TFLITE_MODEL, "wb").write(converted_tflite_model)

91480188

In [ ]:
import datetime
filename = "test_{}.hdf5".format(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S"))
model.save("training_1_15", save_format='tf')

DYNAMIC RANGE QUANTIZATION

In [ ]:
# convert the model to TFLite
!mkdir "tf_models1"
TFLITE_MODEL1 = "tf_models1/dynamic.tflite"

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model('mobilenet_saved_model')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
open(TFLITE_MODEL1, "wb").write(tflite_quant_model)

23160376

FULL INTEGER QUANTIZATION

In [ ]:
def represent_data_gen():
  for ind in range(len(test_subset_generator.filenames)):
    img_with_label=test_subset_generator.next()
    yield[ np.array(img_with_label[0],dtype=np.float32,ndmin=2)]

In [ ]:
# convert the model to TFLite
!mkdir "tf_models2"
TFLITE_MODEL2 = "tf_models2/fullint.tflite"

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model('mobilenet_saved_model')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.represent_data_gen = represent_data_gen
tflite_quant_model = converter.convert()
open(TFLITE_MODEL2, "wb").write(tflite_quant_model)

23160376

FLOAT 16 QUANTIZATION

In [ ]:
# convert the model to TFLite
!mkdir "tf_models3"
TFLITE_MODEL2 = "tf_models3/float16.tflite"

mkdir: cannot create directory ‘tf_models3’: File exists


In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model('mobilenet_saved_model')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()
open(TFLITE_MODEL2, "wb").write(tflite_quant_model)

45749116